In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from alibi.explainers import AnchorTabular
from alibi.datasets import fetch_adult

In [2]:
adult = fetch_adult()
adult.keys()

dict_keys(['data', 'target', 'feature_names', 'target_names', 'category_map'])

In [3]:
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map
print(category_map.keys())
print(category_map)

dict_keys([1, 2, 3, 4, 5, 6, 7, 11])
{1: ['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'], 2: ['Associates', 'Bachelors', 'Doctorate', 'Dropout', 'High School grad', 'Masters', 'Prof-School'], 3: ['Married', 'Never-Married', 'Separated', 'Widowed'], 4: ['?', 'Admin', 'Blue-Collar', 'Military', 'Other', 'Professional', 'Sales', 'Service', 'White-Collar'], 5: ['Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'], 6: ['Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'], 7: ['Female', 'Male'], 11: ['?', 'British-Commonwealth', 'China', 'Euro_1', 'Euro_2', 'Latin-America', 'Other', 'SE-Asia', 'South-America', 'United-States', 'Yugoslavia']}


In [6]:
from alibi.utils import gen_category_map
data_1=gen_category_map(data, categorical_columns=[1, 2, 3, 4, 5, 6, 7, 11])
print(data_1)

{1: [0, 1, 2, 3, 4, 5, 6, 7, 8], 2: [0, 1, 2, 3, 4, 5, 6], 3: [0, 1, 2, 3], 4: [0, 1, 2, 3, 4, 5, 6, 7, 8], 5: [0, 1, 2, 3, 4, 5], 6: [0, 1, 2, 3, 4], 7: [0, 1], 11: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


In [7]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:,:-1]
target = data_perm[:,-1]

idx = 30000
X_train,Y_train = data[:idx,:], target[:idx]
X_test, Y_test = data[idx+1:,:], target[idx+1:]

In [12]:
ordinal_features = [x for x in range(len(feature_names)) if x not in
                    list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())])

In [13]:
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),\
('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [14]:
preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer,
                   ordinal_features),('cat', categorical_transformer,categorical_features)])
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 [0, 8, 9, 10]),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 [1, 2, 3, 4, 5, 6, 7, 11])])

In [15]:
np.random.seed(0)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(preprocessor.transform(X_train), Y_train)

predict_fn = lambda x: clf.predict(preprocessor.transform(x))
print('Train accuracy: ', accuracy_score(Y_train, predict_fn(X_train)))
print('Test accuracy: ', accuracy_score(Y_test, predict_fn(X_test)))

Train accuracy:  0.9655333333333334
Test accuracy:  0.855859375


In [16]:
explainer = AnchorTabular(predict_fn, feature_names, 
                           categorical_names=category_map,seed=1)
explainer.fit(X_train, disc_perc=[25, 50, 75])

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': 1, 'disc_perc': [25, 50, 75]},
  'version': '0.9.0'}
)

In [17]:
idx = 0
class_names = adult.target_names
cls=explainer.predictor(X_test[idx].reshape(1, -1))
print(cls)
print('Prediction: ', class_names[cls[0]])

[0]
Prediction:  <=50K


In [18]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: Marital Status = Separated AND Sex = Female
Precision: 0.95
Coverage: 0.11


In [19]:
idx = 6
class_names = adult.target_names
print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Prediction:  >50K


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


Anchor: Capital Loss > 0.00 AND Relationship = Husband AND Marital Status = Married AND Age > 37.00 AND Race = White AND Sex = Male AND Country = United-States
Precision: 0.72
Coverage: 0.02
